In [1]:
import os, numpy as np, pandas as pd
from glob import glob
from saspt import StateArray, StateArrayDataset, RBME, load_detections
import scipy.io
from traj_sorting import classify_and_write_csv

In [2]:
# Sort trajectories from selected cells and write out to separate csv folders
#
# Naming convention for output files is [FOV number]_[cell number].csv
# Cell number doesn't necessarily go order starting from 1, because it depends on the arbitrary ROI labels assigned by stardist

maskmat = 'out.mat' # mask file (generated using MATLAB cellpicker function)
csvfolder = 'csvFiles' # folder that contains the CSV files from quot tracking 
outfolder = 'sorted_csvs' # folder for sorted CSV output (doesn't have to exist yet)

classify_and_write_csv(maskmat,csvfolder,outfolder)

In [ ]:
# Run SASPT on all sorted cells

sorted_csv_folder = 'sorted_csvs' # where sorted csv files were stored from the previous step
input_files = glob(sorted_csv_folder + '/*.csv')
detections = load_detections(*input_files)
settings = dict(
    likelihood_type = RBME,
    pixel_size_um = 0.16,
    frame_interval = 0.00748,
    focal_depth = 0.7,
    start_frame = 0,
    progress_bar = True,
    sample_size = 100000,
)
SA = StateArray.from_detections(detections, **settings)
print(SA)
print("Trajectory statistics:")
for k, v in SA.trajectories.processed_track_statistics.items():
    print(f"{k : <20} : {v}")
prior_occs = SA.prior_occs
naive_occs = SA.naive_occs
posterior_occs = SA.posterior_occs
marginal_posterior_occs = SA.marginalize_on_diff_coef(posterior_occs)


In [ ]:
SA.occupations_dataframe.to_csv("posterior_occupations.csv", index=False)
SA.plot_occupations("posterior_occupations.png")
SA.plot_assignment_probabilities("assignment_probabilities.png")
SA.plot_temporal_assignment_probabilities("assignment_probabilities_by_frame.png")
import pickle
with open('SA_pickle','wb') as fh:
    pickle.dump(SA,fh)